In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [33]:
iris = load_iris()
X = iris['data']
y = iris['target']
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [3]:
left_nodes = clf.tree_.children_left[clf.tree_.children_left>0]
right_nodes = clf.tree_.children_right[clf.tree_.children_right>0]
node_indicator = clf.decision_path(X)
path_list = []
for i, j in enumerate(X):
    path_list.append(node_indicator.indices[node_indicator.indptr[i]:node_indicator.indptr[i+1]])

## Convert path to strings
path_column = np.array([])
for i, j in enumerate(X):
    path_as_string = []
    for node in path_list[i]:
        if node == 0:
            path_as_string.append('S')
        elif node in left_nodes:
            path_as_string.append('L')
        elif node in right_nodes:
            path_as_string.append('R')
            
    path_as_string.append('E')
    path_as_string = ' '.join(path_as_string)
    path_column = np.append(path_column, path_as_string)

In [64]:
chars = ['S', 'L', 'R', 'E']
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Xnew = np.hstack((X, path_column.reshape(-1,1)))
path_sequence = Xnew[:,4]
data = pd.DataFrame(Xnew)
data[5]=y
df = data.sample(frac=1).reset_index(drop=True)

# prepare dataset for training
get_path_lengths = lambda t: len(t)
paths_lengths = np.array([get_path_lengths(xi) for xi in path_sequence])

In [82]:
vocab_size = 4
label_size = 3
feature_size = 4
# cut the text in semi-redundant sequences of maxlen characters
maxlen = np.max(paths_lengths)
sentences = []
next_chars = []
features = []
labels = []

for i in range(0, len(df)):
    # get the feature
    curr_feat = np.array([df.iloc[i,0:4]])
    curr_path = df.iloc[i,4].split()
    curr_path_len = len(curr_path)
    curr_label = y[i]
    tmp = ''
    for j in range(1,curr_path_len):
        features.append(curr_feat)
        labels.append(curr_label)
        sentences.append(curr_path[0:j])
        next_chars.append(curr_path[j])
print('Vectorization...')

x_sent = np.zeros((len(sentences), maxlen, vocab_size), dtype=np.bool)
x_feat = np.zeros((len(sentences), feature_size), dtype=np.float)
y_chars = np.zeros((len(sentences), len(chars)), dtype=np.bool)
y_feat = np.zeros((len(sentences), label_size), dtype=np.float)
#from keras.utils import to_categorical
#y_feat_tmp = to_categorical(df[5])

Vectorization...


In [84]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x_sent[i, t, char_indices[char]] = 1
    y_chars[i, char_indices[next_chars[i]]] = 1
    x_feat[i,:] = features[i]
    y_feat[i,labels[i]]=1

In [87]:
index = 10
print(y_chars[index],y_feat[index],x_sent[index],x_feat[index])
print(y_chars.shape,y_feat.shape,x_sent.shape,x_feat.shape)

[False  True False False] [1. 0. 0.] [[ True False False False]
 [False False  True False]
 [False  True False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]] [5.5 2.4 3.7 1. ]
(560, 4) (560, 3) (560, 13, 4) (560, 4)


In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

h1_size = 10
h2_size = 5
latent_dim = 10
# Define an input sequence and process it.
input_features = Input(shape=(None, vocab_length))
encoder_state_1 = Dense(h1_size, activation='relu')(input_features)
encoder_state_2 = Dense(h2_size, activation='relu')(encoder_state_1)
output_labels = Dense(1, activation='linear')(encoder_state_2)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(encoder_state_2)
decoder_dense = Dense(vocab_length, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model(input_features, [output_labels,decoder_outputs])
model.summary()

In [109]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Concatenate, concatenate, Flatten

h1_size = 10
latent_dim = 10

input_x_features = Input(shape=(None,feature_size))
hidden_state_x = Dense(h1_size, activation='relu')(input_x_features)
output_labels = Dense(1, activation='linear')(hidden_state_x)

input_sent_features = Input(shape=(maxlen,vocab_size))
lstm_sent = LSTM(latent_dim,input_shape=(maxlen,vocab_size))(input_sent_features)

#hidden_sent = Dense(latent_dim,activation='relu')(lstm_sent)
#flatten_hidden_sent = Flatten(hidden_sent)
#merge_layer = concatenate([hidden_sent,hidden_state_x])
output_chars = Dense(vocab_size, activation='softmax')(lstm_sent)
model = Model([input_x_features,input_sent_features], [output_labels,output_chars])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, None, 4)      0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           (None, 13, 4)        0                                            
__________________________________________________________________________________________________
dense_40 (Dense)                (None, None, 10)     50          input_31[0][0]                   
__________________________________________________________________________________________________
lstm_16 (LSTM)                  (None, 10)           600         input_32[0][0]                   
__________________________________________________________________________________________________
dense_41 (